<a href="https://colab.research.google.com/github/aritejhg/tinyyolov4_person_detector/blob/main/Door_detector_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#collins people at door detector

Objectives:
1. Convert Sunmi and Top View Multiple Person dataset to YOLO format
2. Implement histogram equalisation
3. Get baseline from KDE background subtraction + Historgram
4. Train TinyYOLOv4 model and check performance. Implement with DIoU + Multi-Scale

Result required: Just a indication if people are present in the area. To be passed to other code for talking to MiR



References:
1. [Training tinyYOLOv4](https://colab.research.google.com/drive/1hQO4nOoD6RDxdbz3C1YSiifTsyZjZpYm?usp=sharing)
2. [Top View Multi Person labelled images zip](https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38)
3. [Sunmi dataset](https://github.com/Sunmi-AI-Lab/head-detection-and-tracking/tree/master/datasets) 
4. [DIoU](https://arxiv.org/abs/1911.08287)
5. [Multi-Scale tinyYOLOv4 Research paper](https://link.springer.com/chapter/10.1007/978-3-030-85383-9_1)

In [2]:
#imports 
from google.colab import drive
import os
import shutil

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Download datasets

In [ ]:
#imports
!pip install gdown
import gdown

##Sunmi dataset

In [ ]:
!apt install subversion

In [ ]:
!svn checkout https://github.com/Sunmi-AI-Lab/head-detection-and-tracking/trunk/datasets/in-office /content/Sunmi/


## Top View 


In [ ]:
os.makedirs("/content/TVMP")

In [ ]:
#https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38?usp=sharing
!gdown --folder https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38?usp=sharing /content/TVMP/

In [ ]:
!unzip /content/TVMP/labeled_images/test.zip

In [ ]:
!unzip /content/TVMP/labeled_images/train.zip 

##Backup downloaded datasets to drive

In [ ]:
shutil.copytree("/content/Sunmi","/content/drive/MyDrive/dataset/")

In [ ]:
shutil.copytree("/content/TVMP","/content/drive/MyDrive/dataset/TVMP")

# Converting datasets to YOLO format
While TVMP appears to be in YOLO format, Sunmi is in VOC XML with xmin ymin xmax ymax.

YOLO format is x_center, y_center, width, height. 

Conversion is required for sunmi using script below


In [23]:
# inspired from https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5
import glob
import xml.etree.ElementTree as ET


classes = ["head"]
input_dir = "/content/drive/MyDrive/dataset/in-office/annotations/"
output_dir = "/content/drive/MyDrive/dataset/in-office/labels/"
image_dir = "/content/drive/MyDrive/dataset/in-office/images/"

def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

files = glob.glob(os.path.join(input_dir, '*.xml'))
for i,fil in enumerate(files):
    #print (fil)
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue
    
    result = []
    os.makedirs("/content/drive/MyDrive/dataset/in-office/labels", exist_ok = True)
    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    try: 
      width = int(root.find("size").find("width").text)
      height = int(root.find("size").find("height").text)
    except: 
      width, height = 800, 800
      print(fil)
    print(width,height)

    for obj in root.findall('object'):
        # since we are using only 1 class, this isnt necessary
        # label = obj.find("name").text
        # # check for new classes and append to list
        # if label not in classes:
        #     classes.append(label)
        # index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"0 {bbox_string}")
    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))
        print (f"{i+1}/{len(files)} done")
# the ones with no height width data are giving a negative bbox value when converted to yolo, needs to be checked

800 800
1/1626 done
800 800
2/1626 done
800 800
3/1626 done
800 800
4/1626 done
800 800
5/1626 done
800 800
6/1626 done
800 800
7/1626 done
800 800
8/1626 done
800 800
9/1626 done
800 800
10/1626 done
800 800
11/1626 done
800 800
12/1626 done
800 800
13/1626 done
/content/drive/MyDrive/dataset/in-office/annotations/hide_0173.xml
800 800
14/1626 done
800 800
15/1626 done
800 800
16/1626 done
800 800
17/1626 done
800 800
18/1626 done
800 800
19/1626 done
800 800
20/1626 done
800 800
21/1626 done
800 800
22/1626 done
800 800
23/1626 done
800 800
24/1626 done
800 800
25/1626 done
800 800
26/1626 done
/content/drive/MyDrive/dataset/in-office/annotations/hide_0338.xml
800 800
27/1626 done
800 800
28/1626 done
800 800
29/1626 done
800 800
30/1626 done
800 800
31/1626 done
800 800
32/1626 done
800 800
33/1626 done
800 800
34/1626 done
800 800
35/1626 done
800 800
36/1626 done
/content/drive/MyDrive/dataset/in-office/annotations/hide_0619.xml
800 800
37/1626 done
800 800
38/1626 done
800 800
39

KeyboardInterrupt: ignored

In [8]:
!rm -r /content/drive/MyDrive/dataset/in-office/labels